In [69]:
import pandas as pd
from collections import defaultdict
class Hmm:
    def __init__(self,train):
        self.emission=None # pandas table containing word  states x->y probabitiles and counts
        self.transmission=None  #pandas table containing all state u-> v probabilities and counts
        self.count=None #dictionary containing all u counts
        self.readTrain(train)
    def readTrain(self,train):
        file=open(train,'r',encoding='utf8')
        
        
        xCount=defaultdict(int)
        yseq=['START']
        xyseq=[]
        
        
        for line in file:
            xy=line.split(' ')
            if len(xy)<=1:
                yseq.append('END')
                yseq.append('START')
                continue
            x,y=xy
            x=x.strip()
            y=y.strip()
            xyseq.append([x,y])
            yseq.append(y)
            xCount[x]+=1
        file.close()
        #Emission code
        for i in range(len(xyseq)):
            x,y = xyseq[i]
            if xCount[x]<3:
                xyseq[i] = ['#UNK#',y]
        xyCount = defaultdict(int)
        
        for x,y in xyseq:
            xyCount[x,y]+=1
            
        self.emission= pd.DataFrame([(i[0], i[1], j) for i, j in xyCount.items()],columns=['o','u','Count'])
        del yseq[-1]#delete the last 'start' tag from the sequence
        transDict = defaultdict(int)
        countDict= defaultdict(int)
        for i in range(len(yseq)-1):
            u=yseq[i]
            countDict[u]+=1
            if u=='END':
                continue
            v=yseq[i+1]
            if(u=='START' and v=='END'):#blank line detect
                countDict['START']-=1#remove extra count
                break
            transDict[(u,v)]+=1
        self.transmission=pd.DataFrame([(i[0], i[1], j) for i, j in transDict.items()],columns=['u','v','Count'])
        self.count=pd.DataFrame([(i, j) for i, j in countDict.items()],columns=['u','GCount'])
        self.genprobTransmission()
        self.genprobEmission()
    def genprobTransmission(self):
        self.transmission=pd.merge(self.transmission, self.count, on='u')
        self.transmission['transmission']=self.transmission['Count']/self.transmission['GCount']
    
    def genprobEmission(self):
        self.emission=pd.merge(self.emission, self.count, on='u')
        self.emission['emission']=self.emission['Count']/self.emission['GCount']
        self.emission.drop(['GCount'], axis=1)
        return self.emission
        

In [78]:
EN_transmission = Hmm('./EN/train')
df=EN_transmission.transmission
df
#print(EN_transmission.transmission_proba('B-NP','END'))
#print(EN_transmission.transmission_proba('START','START'))

,u,v,Count,GCount,transmission
0,START,B-NP,4966,7663,0.648049
1,START,B-PP,833,7663,0.108704
2,START,O,1087,7663,0.141850
3,START,B-ADVP,416,7663,0.054287
4,START,B-ADJP,25,7663,0.003262
5,START,B-SBAR,173,7663,0.022576
6,START,B-CONJP,2,7663,0.000261
7,START,B-VP,143,7663,0.018661
8,START,B-INTJ,10,7663,0.001305
9,START,B-LST,8,7663,0.001044
